In [21]:
import datetime as dt
import csv
import pandas as pd
import networkx as nx

In [28]:
vaccine_terms = 'shot|vaccine|vacine|vacines|vaccines|vaccinate|vaccination|vaccinations|vaccinated|vaccinating|vaxxed|vaxx|vax|\
unvaccinated|unvaxxed|antivaxx|antivaccination|anti|\
moderna|pfizer|J&J|immune|immunize|immunizes|immunized|immunization|immunizations'
vaccine_terms_list= vaccine_terms.split('|')

In [ ]:
# use your own path for the data
data_path = '/content/drive/MyDrive/math168_project/data/08_31.csv'

In [48]:
import os
import glob
  
# csv_files = glob.glob(os.path.join(data_path, "*.csv"))

# read the csv file individually
df = pd.read_csv(data_path, dtype="str")
      
# print the location and filename
df = df[['user_id', 'text', 'created_at', 'id', 'in_reply_to_screen_name', 'retweet_screen_name', 'user_screen_name']]
    
# either a reply or a retweet
df = df.dropna(subset=['in_reply_to_screen_name', 'retweet_screen_name'], how='all')
df = df[df.text.str.contains(vaccine_terms)]
    
df['target_user'] = df['in_reply_to_screen_name'].fillna('') + df['retweet_screen_name'].fillna('')
    
# make graph
# G = nx.from_pandas_edgelist(df, 'user_screen_name', 'target_user', create_using=nx.MultiGraph())
    
# save the updated csv to the same directory - decide on a naming convention for these files
# currently it is {original_file_name}_twitter.csv
f_name, _ = os.path.splitext(data_path)
df.to_csv(f_name+'_twitter.csv', index=False, encoding='utf-8-sig')

/var/folders/1w/qqx3tnc943383tv49dm69fd00000gn/T/ipykernel_35662/712566613.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(abs_f_name)
